<a href="https://colab.research.google.com/github/harshalDharpure/Complte-Python-/blob/main/Complete_segmentation_and_Feature_Extraction_in_Lung_Cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Segmentaton Technique

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
img = cv2.imread('/content/input3.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2_imshow(gray)
## Applying dilation for sure_bg detection
ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
## Defining kernel for opening operation
kernel = np.ones((3,3), np.uint8)
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)
cv2_imshow(opening)
## After opening, will perform dilation
sure_bg = cv2.dilate(opening, kernel, iterations=3)
## Sure background image
cv2_imshow(sure_bg)

In [ ]:
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)
sure_fg = np.uint8(sure_fg)


In [ ]:
unknown = np.subtract(sure_bg, sure_fg)
cv2_imshow(unknown)

##Watershed Segmentation Algorithm

In [ ]:
import cv2
import numpy as np
from skimage.feature import peak_local_max
from skimage.segmentation import watershed
from scipy import ndimage

# Load the CT scan image
image = cv2.imread('input3.png', 0)

# Apply Gaussian Blur to the image to remove noise
image = cv2.GaussianBlur(image, (5, 5), 0)

# Obtain the internal marker by thresholding the image
ret, thresh = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Perform morphological closing to fill the small holes in the image
kernel = np.ones((3,3),np.uint8)
closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations = 2)

# Calculate the distance transform
distance = ndimage.distance_transform_edt(closing)

# Obtain the external marker by finding local maxima in the distance transform
local_maxima = peak_local_max(distance, indices=False, footprint=np.ones((3, 3)), labels=closing)
markers, n_markers = ndimage.label(local_maxima)

# Perform marker-controlled watershed segmentation
labels = watershed(-distance, markers, mask=closing)

# Display the segmented image
cv2_imshow(labels)
cv2.waitKey(0)
cv2.destroyAllWindows()


## Global Thresholding Image Segmentation Technique

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# Load the lung CT scan image
img = cv2.imread('input3.png', 0)

# Apply a global threshold to the image
ret, thresh = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY)

# Display the original and thresholded images
cv2_imshow(img)
cv2_imshow(thresh)

# Wait for a key press
cv2.waitKey(0)
cv2.destroyAllWindows()


## Region & Edge based segmentation

In [ ]:
from skimage.color import rgb2gray
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import ndimage
from PIL import Image
from sklearn.cluster import KMeans
from skimage.filters import sobel
import skimage
import warnings
warnings.filterwarnings("ignore")

In [ ]:
image=Image.open('input3.png')
image=image.resize((320,225))
image=np.array(image)
plt.imshow(image)

##Region Based Segmentation of image with K=2 Folds

In [ ]:
import cv2
import numpy as np

# Load the input image
img = cv2.imread('input3.png')

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply median filtering to remove noise
gray = cv2.medianBlur(gray, 5)

# Set the number of iterations for erosion and dilation
iterations = 5

# Initialize the mask
mask = np.zeros_like(gray)

# Get the width and height of the image
width, height = gray.shape

# Divide the image into K=2 folds
for k in range(2):

    # Set the ROI for the current fold
    if k == 0:
        roi = (0, 0, height // 2, width)
    else:
        roi = (height // 2, 0, height, width)

    # Get the ROI of the image
    roi_gray = gray[roi[0]:roi[2], roi[1]:roi[3]]

    # Compute the threshold using Otsu's method
    _, thresh = cv2.threshold(roi_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Perform erosion and dilation to remove small holes
    for i in range(iterations):
        thresh = cv2.erode(thresh, None, iterations=1)
        thresh = cv2.dilate(thresh, None, iterations=1)

    # Add the thresholded ROI to the mask
    mask[roi[0]:roi[2], roi[1]:roi[3]] = thresh

# Apply the mask to the input image
segmented = cv2.bitwise_and(img, img, mask=mask)

# Display the original and segmented images
cv2_imshow(img)
cv2_imshow(segmented)
cv2.waitKey(0)
cv2.destroyAllWindows()


## Region Based Segmentation of Image with K=5 Folds

In [ ]:
arr=gray.flatten()
for i in range(len(arr)):
    if arr[i]>=arr.mean():
        arr[i]=4
    elif arr[i]>=0.75:
        arr[i]=3
    elif arr[i]>0.5 :
        arr[i]=2
    elif arr[i]>0.25:
        arr[i]=1
    else:
        arr[i]=0
gray_segmented_2=arr.reshape(gray.shape[0],gray.shape[1])

In [ ]:
plt.figure(figsize=(18,8))
plt.imshow(gray_segmented_2,cmap='pink')
plt.axis("off")
plt.show()

## Region-based method using the watershed transform

In [ ]:
imm=image[:,:,0]
elevation_map = sobel(imm)

fig, ax = plt.subplots(figsize=(18,8))
ax.imshow(elevation_map, cmap='gray', interpolation='nearest')
ax.axis('off')
ax.set_title('elevation_map')
plt.show()

In [ ]:
plt.hist(imm.flatten())
plt.show()

In [ ]:
markers = np.zeros_like(imm)
markers[imm < 117] = 1
markers[imm > 232] = 2

fig, ax = plt.subplots(figsize=(8,4))
ax.imshow(markers, cmap='Spectral', interpolation='nearest')
ax.axis('off')
ax.set_title('markers')

## Edge-Based Segmentation Technique

In [ ]:
kernel_laplace = np.array([np.array([1, 1, 1]), np.array([1, -8, 1]), np.array([1, 1, 1])])

out_l = ndimage.convolve(gray_segmented_2, kernel_laplace, mode='reflect')
arr=[]
for i in out_l.flatten():
    if i == 0:
        arr.append(0)
    else:
        arr.append(-100)
arr=np.array(arr)
arr=arr.reshape(out_l.shape[0],out_l.shape[1])
plt.imshow(arr,cmap='gray', interpolation='nearest')

In [ ]:
im=image/255
pic=im.reshape(im.shape[0]*im.shape[1],im.shape[2])

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(16, 8))
count=1
for i in range(2):
    for j in range(3):

        kmeans = KMeans(n_clusters=count+1, random_state=0).fit(pic)
        pic_print = kmeans.cluster_centers_[kmeans.labels_]
        clustered_pic=pic_print.reshape(im.shape[0],im.shape[1],im.shape[2])
        count+=1
        ax[i][j].set_title('Cluster '+str(count))
        ax[i][j].imshow(clustered_pic)
        ax[i][j].set_axis_off()
plt.show()

## Edge Based Detection

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
# read image
img = cv2.imread('input3.png', cv2.IMREAD_GRAYSCALE)

# apply Gaussian blur to reduce noise
img_blur = cv2.GaussianBlur(img, (3,3), 0)

# apply Canny edge detector
edges = cv2.Canny(img_blur, 50, 150)

# perform morphological closing to connect broken edges
kernel = np.ones((5,5),np.uint8)
closing = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

# display results
cv2_imshow(img)
cv2_imshow(edges)
cv2_imshow(closing)
cv2.waitKey(0)
cv2.destroyAllWindows()


## K-means Clustering Algorithm

In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from google.colab.patches import cv2_imshow
# Load the input image
image = cv2.imread('input3.png')

# Reshape the image to a 2D array of pixels
pixels = image.reshape((-1, 3))

# Define the number of clusters
n_clusters = 3

# Apply K-means clustering
kmeans = KMeans(n_clusters=n_clusters)
kmeans.fit(pixels)
labels = kmeans.predict(pixels)

# Reshape the labels back to the original image shape
labels = labels.reshape(image.shape[:2])

# Create a mask for each label
masks = [(labels == i).astype(np.uint8) for i in range(n_clusters)]

# Display the output
for i, mask in enumerate(masks):
    cv2_imshow(mask * 255)

cv2.waitKey(0)
cv2.destroyAllWindows()
